# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
data_file = "../output_data/output.csv" 

weather_data = pd.read_csv(data_file)
weather_data.drop(columns=["Unnamed: 0"], inplace=True)
weather_data.head()

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Bluff,NZ,1612840515,-46.6000,168.3333,54.52,90,100,12.68
1,Yangi Marg`ilon,UZ,1612840369,40.4272,71.7189,44.60,87,75,4.61
2,Atuona,PF,1612840515,-9.8000,-139.0333,79.56,78,99,18.81
3,Hobart,AU,1612840229,-42.8794,147.3294,62.01,58,75,8.05
4,Mataura,NZ,1612840315,-46.1927,168.8643,53.78,96,100,4.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [6]:
locations = weather_data[["Latitude", "Longitude"]]

humidity  = weather_data["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=5)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Ideal Vacation Spot: 
vacation_spots = weather_data.loc[weather_data['Max Temperature'] < 85]
vacation_spots = vacation_spots[vacation_spots['Max Temperature'] > 70]
vacation_spots.head()

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
2,Atuona,PF,1612840515,-9.8000,-139.0333,79.56,78,99,18.81
11,Bafoulabé,ML,1612840516,13.8065,-10.8321,72.39,20,5,5.08
12,Saint-Philippe,RE,1612840516,-21.3585,55.7679,77.00,50,0,8.05
17,Butaritari,KI,1612840292,3.0707,172.7902,82.06,82,60,21.09
18,Vila Velha,BR,1612840315,-20.3297,-40.2925,77.00,94,90,3.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
def call_places(latitude, longitude, radius, searchType, keyword, key):
    call_string = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={radius}&type={searchType}&keyword={keyword}&key={key}"
    return call_string

In [9]:
hotel_df = pd.DataFrame(vacation_spots, columns=["City", "Country", "Latitude", "Longitude"])
hotel_df["Hotel Name"] = ""
hotel_df.columns = ['City', 'Country', 'Lat', 'Lng', 'Hotel Name']
hotel_df.reset_index(inplace=True)
hotel_df

,index,City,Country,Lat,Lng,Hotel Name
0,2,Atuona,PF,-9.8000,-139.0333,
1,11,Bafoulabé,ML,13.8065,-10.8321,
2,12,Saint-Philippe,RE,-21.3585,55.7679,
3,17,Butaritari,KI,3.0707,172.7902,
4,18,Vila Velha,BR,-20.3297,-40.2925,
...,...,...,...,...,...,...
166,537,Cockburn Town,TC,21.4612,-71.1419,
167,544,Rio Grande City,US,26.3798,-98.8203,
168,553,Boyolangu,ID,-8.1181,111.8935,
169,557,Matanzas,CU,23.0411,-81.5775,


In [10]:
hotel_name = []
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    url = call_places(lat,lng,5000,'hotel','hotel',g_key)
    # make request and print url
    name_address = requests.get(url)
    # convert to json
    name_address = name_address.json()
    try:

        hotel_name.append(name_address['results'][0]['name'])
    except:
        hotel_name.append('No Hotel')
hotel_name

['No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',
 'No Hotel',

In [11]:
hotel_df['Hotel Name']= hotel_name
hotel_df = hotel_df.dropna()
hotel_df.to_csv('../output_data/hotel_export.csv')

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))